In [143]:
#pip install -U gymnasium
#pip install stable-baselines3

In [144]:
from typing import Optional
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import torch
from datetime import datetime
import pytz

In [145]:
# can change this as needed. use as a placeholder for now
# timestamp, bidprice1, bidvol1, bidprice2, bidvol2, bidprice3, bidvol3, askprice1, askvol1, askprice2, askvol2, askprice3, askvol3, OFI (order flow imbalance. volume of buys vs sells. between -1 and 1) of last window, volume weighted price of last window, insitutional vs retail trade imbalance
# institutional trades are typically done in volumes of 100. count volume of trades with % 100 = 0
# forumla: (institutional_vol - (sum(trades_vol) - institutional_vol)) / (sum(trades_vol))
# if institutional_vol = 500 and trades_vol = 500. (500 - 0) / 500 = 1
# if institutional_vol = 250 and trades_vol = 500. (250 - 250) / 500 = 0
# if institutional_vol = 0 and trades_vol = 500. (0 - 500) / 500 = -1

example_LOB = torch.tensor([
    [1772202600, 55.8, 2, 55.76, 6, 55.7, 4, 55.82, 8, 55.88, 2, 55.89, 4, 0.8, 55.78, 0],
    [1772202601, 55.78, 10, 55.73, 12, 55.71, 8, 55.81, 8, 55.84, 12, 55.87, 8, 0.65, 55.83, 1],
    [1772202602, 55.78, 10, 55.74, 3, 55.73, 12, 55.81, 8, 55.84, 12, 55.89, 4, 0.6, 55.82, 1],
    [1772202603, 55.76, 4, 55.75, 2, 55.74, 8, 55.81, 8, 55.82, 2, 55.84, 11, 0.82, 55.77, 0],
    [1772202604, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202605, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202606, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202607, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202608, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202609, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202610, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202611, 55.78, 10, 55.73, 12, 55.71, 8, 55.81, 8, 55.84, 12, 55.87, 8, 0.65, 55.83, 1],
    [1772202612, 55.78, 10, 55.74, 3, 55.73, 12, 55.81, 8, 55.84, 12, 55.89, 4, 0.6, 55.82, 1],
    [1772202613, 55.76, 4, 55.75, 2, 55.74, 8, 55.81, 8, 55.82, 2, 55.84, 11, 0.82, 55.77, 0],
    [1772202614, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202615, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202616, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202617, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202618, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202619, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202620, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202621, 55.78, 10, 55.73, 12, 55.71, 8, 55.81, 8, 55.84, 12, 55.87, 8, 0.65, 55.83, 1],
    [1772202622, 55.78, 10, 55.74, 3, 55.73, 12, 55.81, 8, 55.84, 12, 55.89, 4, 0.6, 55.82, 1],
    [1772202623, 55.76, 4, 55.75, 2, 55.74, 8, 55.81, 8, 55.82, 2, 55.84, 11, 0.82, 55.77, 0],
    [1772202624, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202625, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202626, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202627, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202628, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202629, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772202630, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
], dtype=torch.float64)

In [146]:
example_LOB.shape

torch.Size([31, 16])

In [147]:
def embed_positions(x, time_scale):
    sin_pos = np.sin(2 * np.pi * x / time_scale)
    cos_pos = np.cos(2 * np.pi * x / time_scale)
    return sin_pos, cos_pos

In [148]:
fee_pct = 0.0005

class LOBEnv(gym.Env):
    def __init__(self, orderbook, device, book_depth):
        # large_cpu_tensor should be pinned for speed
        if device == 'cuda':
            self.data = orderbook.pin_memory() 
        else:
            self.data = orderbook
        self.device = device
        self.book_depth = book_depth
        self.current_step = 0
        self.max_steps = self.data.shape[0] - 1
        self.initial_cash = 100_000.0
        
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(low=-float('inf'), high=float('inf'), shape=(46,), dtype=float)

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.market_open_step = 0
        self.market_open_time = None
        self.time_since_open = 0

        self.start_year = 2022 # change this to whatever year testing starts at
        self.current_year = None
        self.weekday = None
        self.current_month = None
        self.current_date = None
        self.current_ts = 0
        
        self.cash = self.initial_cash
        self.total_balance = self.initial_cash
        self.inventory = 0
        self.entry_price = 0
        self.steps_held = 0
        self.transaction_costs = 0
        self.total_return = 0

        self.volume_mismatches = 0
        
        #return self.data[self.current_step].to(self.device, non_blocking=True), {}
        return self._get_obs(), {}
    
    def _get_LOB_data(self, LOB):
        bid_prices = LOB[1:(self.book_depth*2)+1:2]
        bid_volumes = LOB[2:(self.book_depth*2)+1:2]

        ask_prices = LOB[(self.book_depth*2)+1:(self.book_depth*4)+1:2]
        ask_volumes = LOB[(self.book_depth*2)+2:(self.book_depth*4)+1:2]

        return bid_prices, bid_volumes, ask_prices, ask_volumes
    
    def _update_time_info(self, current_ts, current_date):
        year, hour, minute, second = current_date.year, current_date.hour, current_date.minute, current_date.second
        # if new day
        if hour == 9 and minute == 30 and second == 0:
            self.current_date = current_date
            self.market_open_time = current_ts
            self.market_open_step = self.current_step

            self.current_year = year
            self.current_month = self.current_date.month
            self.weekday = current_date.weekday()
        
        self.current_ts = current_ts.item()
        self.time_since_open = current_ts.item() - self.market_open_time

    
    def _create_pos_encodings(self):
        # 60 * 60 * 24 = total seconds in a day
        daily_pos_sin, daily_pos_cos = embed_positions(self.time_since_open.item(), 60*60*24)
        weekday_pos_sin, weekday_pos_cos = embed_positions(self.current_date.weekday(), 7)
        monthly_pos_sin, monthly_pos_cos = embed_positions(self.current_month, 12)
        year_pos = self.current_year - self.start_year

        return daily_pos_sin, daily_pos_cos, weekday_pos_sin, weekday_pos_cos, monthly_pos_sin, monthly_pos_cos, year_pos
    
    def _get_historical_info(self):
        index_10_sec_ago = max(self.current_step - 10, self.market_open_step)
        index_1_min_ago = max(self.current_step - 60, self.market_open_step)
        index_5_min_ago = max(self.current_step - 300, self.market_open_step)
        
        LOB_10s = self.data[index_10_sec_ago]
        LOB_1m = self.data[index_1_min_ago]
        LOB_5m = self.data[index_5_min_ago]

        midpoint_10s = (LOB_10s[7] + LOB_10s[1]) / 2
        midpoint_1m = (LOB_1m[7] + LOB_1m[1]) / 2
        midpoint_5m = (LOB_5m[7] + LOB_5m[1]) / 2

        midpoints = torch.stack([midpoint_10s, midpoint_1m, midpoint_5m])

        print(midpoints)
        return midpoints
    
    def _get_basic_obs(self, LOB):
        bid_prices, bid_volumes, ask_prices, ask_volumes = self._get_LOB_data(LOB)

        current_ts = LOB[0]
        current_date = datetime.fromtimestamp(current_ts.item(), pytz.timezone("US/Eastern"))
        self._update_time_info(current_ts, current_date)

        return bid_prices, bid_volumes, ask_prices, ask_volumes

        
    def _get_obs(self):
        LOB = self.data[self.current_step]
        
        # get basic observational data from LOB
        bid_prices, bid_volumes, ask_prices, ask_volumes = self._get_basic_obs(LOB)

        cumu_bid_volume = sum(bid_volumes)
        cumu_ask_volume = sum(ask_volumes)
        bid_liquidity = sum(bid_prices * bid_volumes)
        ask_liquidity = sum(ask_prices * ask_volumes)
        # create features from the orderbook
        order_book_imbalance = (cumu_bid_volume - cumu_ask_volume) / (cumu_bid_volume + cumu_ask_volume)
        spread = LOB[7] - LOB[1]
        midpoint = (LOB[7] + LOB[1]) / 2
        microprice = (bid_liquidity + ask_liquidity) / (cumu_bid_volume + cumu_ask_volume)

        # get trade info
        trade_info = LOB[-3:]

        # create positonal embeddings
        daily_pos_sin, daily_pos_cos, weekday_pos_sin, weekday_pos_cos, monthly_pos_sin, monthly_pos_cos, year_pos = self._create_pos_encodings()

        # get historical data
        historical_midpoints = self._get_historical_info()
        midpoint_changes = midpoint / historical_midpoints

        # normalize existing values
        book_prices_log_norm = torch.cat([ask_prices / midpoint, bid_prices / midpoint])
        book_volumes_norm = torch.cat([ask_volumes / cumu_ask_volume, bid_volumes / cumu_bid_volume])
        book_values_log = torch.cat([
            ask_prices,
            bid_prices,
            ask_volumes,
            bid_volumes,
            cumu_ask_volume.view(1),
            cumu_bid_volume.view(1),
            ask_liquidity.view(1),
            bid_liquidity.view(1),
            midpoint.view(1),
            microprice.view(1)
        ])

        book_prices_log_norm = torch.log1p(book_prices_log_norm)
        book_values_log = torch.log(book_values_log)
        trade_info[1] = torch.log(trade_info[1])

        LOB_norm = torch.cat([book_prices_log_norm, book_volumes_norm, book_values_log, trade_info])

        # create tensors that will be used for the observation space
        # create tensor private state from inventory and balance
        private_state = torch.tensor([
            self.cash / self.initial_cash,
            np.log1p(self.inventory),
            self.total_balance / self.initial_cash
            
        ], dtype=torch.float32)

        # create tensor for engineed features
        '''engineered_features = torch.stack([
            cumu_bid_volume, cumu_ask_volume, bid_liquidity, ask_liquidity, order_book_imbalance, spread, midpoint, microprice
        ])'''

        # create tensor for positional embeddings
        positional_embeddings = torch.tensor([
            daily_pos_sin, daily_pos_cos, weekday_pos_sin, weekday_pos_cos, monthly_pos_sin, monthly_pos_cos, year_pos
        ])

        # combine all tensors together
        #full_obs = torch.cat([LOB, engineered_features, private_state, positional_embeddings])
        full_obs = torch.cat([LOB_norm, midpoint_changes, private_state, positional_embeddings])
        if self.device == 'cuda':
            full_obs = full_obs.to(device=self.device, dtype=torch.float32, non_blocking=True)
        else:
            full_obs = full_obs.to(device=self.device, dtype=torch.float32)
        print(f'observation space shape: {full_obs.shape}')
        return full_obs

    def step(self, action):
        # 1. get LOB data at current state and make necessary variables for the action taken
        LOB = self.data[self.current_step]

        bid_prices, bid_volumes, ask_prices, ask_volumes = self._get_basic_obs(LOB)

        exec_shares = 0
        exec_cost = 0
        exec_fees = 0

        if action == 1: # buy order
            trade_budget = self.cash * 0.01 # 1% of portfolio value
            remaining_budget = trade_budget
            
            for i in range(len(ask_prices)):
                price = ask_prices[i].item()
                volume = ask_volumes[i].item()

                # get the maximum number of shares that can be traded based on budget
                max_shares = remaining_budget / (price * (1 + fee_pct))

                # take the minimum volume between what is available and wehat we can trade
                shares_taken = min(volume, max_shares)

                # compute metrics
                cost = shares_taken * price
                fees = cost * fee_pct
                exec_shares += shares_taken
                exec_cost += (cost + fees)
                exec_fees += fees
                remaining_budget -= (cost + fees)

                if remaining_budget <= 0 or shares_taken < volume:
                    break

            if exec_shares > 0:
                total_val = (self.inventory * self.entry_price) + exec_cost
                self.inventory += exec_shares
                self.entry_price = total_val / self.inventory
                self.cash -= exec_cost
                self.transaction_costs += exec_fees
            
        elif action == 2: # sell order
            trade_vol = self.inventory
            remaining_inventory = trade_vol

            for i in range(len(bid_prices)):
                price = bid_prices[i].item()
                volume = bid_volumes[i].item()

                # get the max number of shares that can be sold
                shares_sold = min(volume, remaining_inventory)

                # compute metrics
                cost = shares_sold * price
                fees = cost * fee_pct
                exec_shares += shares_sold
                exec_cost += (cost - fees)
                exec_fees += fees
                remaining_inventory -= shares_sold

                if remaining_inventory <= 0 or shares_sold < volume:
                    break
            # handle edge case where not all shares are sold. give warning and execute remaining sale at the worst bid price
            if exec_shares > 0:
                self.total_return += exec_cost - (self.entry_price * self.inventory)
                self.inventory = 0
                self.entry_price = 0
                self.cash += exec_cost
                self.transaction_costs += exec_fees
                
        # 2. Increment step
        self.current_step += 1

        if self.current_step >= self.max_steps:
            terminated = True
            truncated = False
        elif self.total_balance <= 0:
            terminated = False
            truncated = True 
        else:
            terminated = False
            truncated = False
        obs = self._get_obs()

        # replace these variables with the variables from the observation space
        midpoint = (obs[6].item() + obs[0].item()) / 2
        new_total_balance = self.cash + (self.inventory * midpoint)
        reward = new_total_balance - self.total_balance
        self.total_balance = new_total_balance

        return obs, reward, terminated, truncated, {}

In [149]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
torch.set_printoptions(precision=6, sci_mode=False)
env = LOBEnv(example_LOB, device, 3)

obs, info = env.reset()
print(obs)

for i in range(env.max_steps + 1):
    obs, reward, terminated, truncated, info = env.step(0)
    #print(obs)

    if terminated or truncated:
        break

tensor([55.810000, 55.810000, 55.810000], dtype=torch.float64)
observation space shape: torch.Size([46])
tensor([ 0.693237,  0.693774,  0.693864,  0.693058,  0.692699,  0.692161,
         0.571429,  0.142857,  0.285714,  0.166667,  0.500000,  0.333333,
         4.022132,  4.023207,  4.023386,  4.021774,  4.021057,  4.019980,
         2.079442,  0.693147,  1.386294,  0.693147,  1.791759,  1.386294,
         2.639057,  2.484907,  6.661701,  6.505724,  4.021953,  4.021801,
         0.800000,  4.021415,  0.000000,  1.000000,  1.000000,  1.000000,
         1.000000,  0.000000,  1.000000,  0.000000,  1.000000, -0.433884,
        -0.900969,  0.866025,  0.500000,  4.000000], device='mps:0')
tensor([55.810000, 55.810000, 55.810000], dtype=torch.float64)
observation space shape: torch.Size([46])
tensor([55.810000, 55.810000, 55.810000], dtype=torch.float64)
observation space shape: torch.Size([46])
tensor([55.810000, 55.810000, 55.810000], dtype=torch.float64)
observation space shape: torch.Size

In [150]:
env.action_space

Discrete(3)

In [151]:
#LOB = example_LOB[0]
#[LOB[i] for i in range(0, 6, 2)]
book_depth = 3
bid_prices = torch.tensor([example_LOB[0][i] for i in range(0, book_depth*2, 2)])
bid_volumes = torch.tensor([example_LOB[0][i] for i in range(1, book_depth*2, 2)])

ask_prices = torch.tensor([example_LOB[0][i] for i in range(book_depth*2, book_depth*4, 2)])
ask_volumes = torch.tensor([example_LOB[0][i] for i in range((book_depth*2)+1, book_depth*4, 2)])

print(bid_prices, bid_volumes)
print(sum(bid_volumes).item())
#print(sum(bid_prices * bid_volumes).item())

print(ask_prices, ask_volumes)
print(sum(ask_volumes).item())

cumu_bid_volume = sum(bid_volumes)
cumu_ask_volume = sum(ask_volumes)
orderbook_imbalance = (cumu_bid_volume - cumu_ask_volume) / (cumu_bid_volume + cumu_ask_volume)

print(orderbook_imbalance)

spread = example_LOB[0][6] - example_LOB[0][0]
print(spread)

bid_liquidity = sum(bid_prices * bid_volumes)
ask_liquidity = sum(ask_prices * ask_volumes)

print(bid_liquidity)
print(ask_liquidity)

tensor([1772202600.,          2.,          6.], dtype=torch.float64) tensor([55.800000, 55.760000, 55.700000], dtype=torch.float64)
167.26
tensor([4., 8., 2.], dtype=torch.float64) tensor([55.820000, 55.880000, 55.890000], dtype=torch.float64)
167.59
tensor(-0.000986, dtype=torch.float64)
tensor(-1772202596., dtype=torch.float64)
tensor(98888905525.720001, dtype=torch.float64)
tensor(782.100000, dtype=torch.float64)


In [152]:
max_trade_amount = 100_000 * 0.01 # 1% of portfolio value
max_trade_amount

1000.0

In [153]:
print(example_LOB[0][0:6:2])
print(example_LOB[0][1:6:2])

tensor([1772202600.,          2.,          6.], dtype=torch.float64)
tensor([55.800000, 55.760000, 55.700000], dtype=torch.float64)


In [154]:
bid_prices = example_LOB[0][book_depth*2:book_depth*4:2]
bid_volumes = example_LOB[0][(book_depth*2)+1:book_depth*4:2]

In [155]:
print(bid_prices)
print(bid_volumes)

tensor([4., 8., 2.], dtype=torch.float64)
tensor([55.820000, 55.880000, 55.890000], dtype=torch.float64)


In [156]:
torch.tensor([1,])

tensor([1])

In [157]:
microprice = (bid_liquidity + ask_liquidity) / (cumu_bid_volume + cumu_ask_volume)
microprice

tensor(295322999.276751, dtype=torch.float64)